In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import time

### **Data Reading**

In [0]:
df = spark.read.format("parquet")\
  .load("abfss://bronze@avidatabricksete.dfs.core.windows.net/customers")

In [0]:
df.display()

In [0]:
df = df.drop("_rescued_data")
df.display()

In [0]:
df = df.withColumn("email_domain", split(col("email"), "@")[1])
df.display()

In [0]:
df.groupBy("email_domain").agg(count("customer_id").alias("total_customers")).sort("total_customers", ascending=False).display()

In [0]:
df_gmail = df.filter(col("email_domain") == "gmail.com")
df_gmail.display()
time.sleep(5)
df_yahoo = df.filter(col("email_domain")=="yahoo.com")
df_yahoo.display()
time.sleep(5)
df_hotmail = df.filter(col("email_domain")=="hotmail.com")
df_hotmail.display()
time.sleep(5)

In [0]:
df = df.withColumn("full_name",concat(col("first_name"),lit(" "),col("last_name")))
df = df.drop("first_name","last_name")
df.display()

In [0]:
df.write.mode("overwrite").format("delta").save("abfss://silver@avidatabricksete.dfs.core.windows.net/customers")

In [0]:
df = spark.read.format("delta").load("abfss://silver@avidatabricksete.dfs.core.windows.net/customers")
df.display()

In [0]:
%sql
create schema databrciks_catalog.silver

In [0]:
%sql
DROP TABLE IF EXISTS databrciks_catalog.silver.customers_silver;

CREATE TABLE databrciks_catalog.silver.customers_silver
USING DELTA
LOCATION 'abfss://silver@avidatabricksete.dfs.core.windows.net/customers';



In [0]:
%sql
select * from databrciks_catalog.silver.customers_silver